In [ ]:
import pandas as pd
all_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_pull_request.parquet")
all_repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")
all_user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_user.parquet")

In [ ]:

# Ensure created_at is datetime
all_pr_df['created_at'] = pd.to_datetime(all_pr_df['created_at'])

# Count agent PRs per repo
pr_counts = all_pr_df.groupby('repo_id').size().reset_index(name='agent_pr_count')

# Filter for repos with >= 1000 agent PRs
large_repos = pr_counts[pr_counts['agent_pr_count'] >= 100]

# Get first agent PR date per repo
first_pr_dates = all_pr_df.groupby('repo_id')['created_at'].min().reset_index(name='first_pr_date')

# Merge counts and first date
result = large_repos.merge(first_pr_dates, on='repo_id')

# Merge with repo metadata for repo name and URL
result = result.merge(all_repo_df, left_on='repo_id', right_on='id')

# Select relevant columns
result = result[['full_name', 'url', 'agent_pr_count', 'first_pr_date']]

# Sort by number of agent PRs descending
result = result.sort_values(by='agent_pr_count', ascending=False)

# Take top 100 repos


# Save to TXT
with open("top_agent_repos.txt", "w", encoding="utf-8") as f:
    for idx, row in result.iterrows():
        f.write(f"{row['full_name']} | {row['url']} | {row['agent_pr_count']} agent PRs | first PR: {row['first_pr_date'].date()}\n")

print("Saved top 100 agent repos to top_agent_repos.txt")

In [ ]:
import pandas as pd

# Ensure created_at is datetime
all_pr_df['created_at'] = pd.to_datetime(all_pr_df['created_at'])

# Count agent PRs per repo
pr_counts = all_pr_df.groupby('repo_id').size().reset_index(name='agent_pr_count')

# Filter for repos with >= 1000 agent PRs
large_repos = pr_counts[pr_counts['agent_pr_count'] >= 100]

# Get first agent PR date per repo
first_pr_dates = all_pr_df.groupby('repo_id')['created_at'].min().reset_index(name='first_pr_date')

# Merge counts and first date
result = large_repos.merge(first_pr_dates, on='repo_id')

# Merge with repo metadata for repo name and URL
result = result.merge(all_repo_df, left_on='repo_id', right_on='id')

# Select relevant columns
result = result[['full_name', 'url', 'agent_pr_count', 'first_pr_date']]

# Sort by number of agent PRs descending
result = result.sort_values(by='agent_pr_count', ascending=False)

# Take top 100 repos


# Save to TXT
with open("top_agent_repos.txt", "w", encoding="utf-8") as f:
    for idx, row in result.iterrows():
        f.write(f"{row['full_name']} | {row['url']} | {row['agent_pr_count']} agent PRs | first PR: {row['first_pr_date'].date()}\n")

print("Saved top 100 agent repos to top_agent_repos.txt")